In [1]:
import os
import cv2
import numpy as np
from sklearn.decomposition import PCA
from keras.preprocessing.image import ImageDataGenerator

data_dir = r"C:\Users\Dell/Downloads/archive (2)/train"
image_size = (224, 224)

num_components = 100
images = []

datagen = ImageDataGenerator(rescale=1./255)
image_generator = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=32,  
    class_mode=None,  
    shuffle=False  
)

for _ in range(image_generator.samples // image_generator.batch_size):
    batch = image_generator.next()
    images.extend(batch)
images = np.array(images)

images_flattened = images.reshape(images.shape[0], -1)

pca = PCA(n_components=num_components)
pca.fit(images_flattened)
pca_images = pca.transform(images_flattened)
labels = ["Normal"] * 1341 + ["Pneumonia"] * 3875

print("Shape of pca_images:", pca_images.shape)
print("Number of labels:", len(labels))

Found 5216 images belonging to 2 classes.
Shape of pca_images: (5216, 100)
Number of labels: 5216


In [7]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

x_train, x_test, y_train, y_test = train_test_split(pca_images, labels, test_size=0.2, random_state=42)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
xgb_classifier = XGBClassifier(random_state=42)
svm_classifier = SVC(kernel='linear', C=1.0)
knn_classifier = KNeighborsClassifier(n_neighbors=5)
base_classifiers = [('rf', rf_classifier),('svm', svm_classifier), ('knn', knn_classifier), ('xgb', xgb_classifier)]
stack_classifier = StackingClassifier(estimators=base_classifiers, final_estimator=RandomForestClassifier(n_estimators=100, random_state=42))
stack_classifier.fit(x_train, y_train)
y_pred = stack_classifier.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Stacked Classifier Accuracy: {accuracy:.2f}")

Stacked Classifier Accuracy: 0.96


In [9]:
import joblib

model_filename = 'stack_classifier_model.joblib'
joblib.dump(stack_classifier, model_filename)
print(f"Stack Classifier model saved as '{model_filename}'")

Stack Classifier model saved as 'stack_classifier_model.joblib'


In [22]:
from PIL import Image
import numpy as np
img_path = r"C:\Users\Dell\Downloads\archive (2)\train\normal\IM-0125-0001.jpeg"
img = Image.open(img_path)
img = img.convert('L')
img = img.resize((224, 224))
img_array = np.array(img)
img_vector = img_array.reshape(-1)
from sklearn.decomposition import PCA
num_components = min(100, img_vector.size)  
pca = PCA(n_components=num_components)
pca_image_vector = pca.fit_transform(img_array)
import joblib
model_filename = 'stack_classifier_model.joblib'  
stack_classifier = joblib.load(model_filename)
predictions = stack_classifier.predict(pca_image_vector)
print(predictions[0])



Normal
